<a href="https://colab.research.google.com/github/SoheilBadri2000/DataScience2/blob/main/45B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Real-Time Voice Cloning

In [1]:
!pip install torch
!git clone https://github.com/CorentinJ/Real-Time-Voice-Cloning.git
%cd Real-Time-Voice-Cloning
!pip install numpy==1.22.3

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [3]:
# remove numpy from req file before run
%cd Real-Time-Voice-Cloning
!pip install -r requirements.txt

[Errno 2] No such file or directory: 'Real-Time-Voice-Cloning'
/content/Real-Time-Voice-Cloning
  Using cached inflect-5.3.0-py3-none-any.whl (32 kB)
  Using cached librosa-0.8.1-py3-none-any.whl (203 kB)
  Using cached matplotlib-3.5.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.9 MB)
  Using cached Pillow-8.4.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Using cached PyQt5-5.15.6-cp36-abi3-manylinux1_x86_64.whl (8.3 MB)
  Using cached scikit_learn-1.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.5 MB)
  Using cached scipy-1.7.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (39.9 MB)
  Using cached sounddevice-0.4.3-py3-none-any.whl (31 kB)
  Using cached SoundFile-0.10.3.post1-py2.py3-none-any.whl (21 kB)
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)
  Using cached umap_learn-0.5.2-py3-none-any.whl
  Using cached Unidecode-1.3.2-py3-none-any.whl (235 kB)
  Using cached urllib3-1.26.7-py2.py3-none-

In [1]:
import sys
from pathlib import Path
import librosa
from IPython.display import Audio
import soundfile as sf
import numpy as np

In [2]:
sys.path.append("Real-Time-Voice-Cloning/")
!mkdir -p encoder/saved_models
!mkdir -p synthesizer/saved_models
!mkdir -p vocoder/saved_models

!gdown --id "1hoFzOhkAA_FMLxZujFma00Cj4cR1fmTL" -O encoder/saved_models/encoder.pt
!gdown --id "19DGdDixWUAzzF88aNd1FL3wQngtzdnry" -O synthesizer/saved_models/synthesis.pt
!gdown --id "1lc--7GQqlL7uJ5GC2ulJL4H6n9mZF4Pq" -O vocoder/saved_models/vocoder.pt

from encoder import inference as encoder
from synthesizer.inference import Synthesizer
from vocoder import inference as vocoder

# Load models (paths might need to be adjusted based on where you've stored them)
encoder.load_model(Path("encoder/saved_models/encoder.pt"))
synthesizer = Synthesizer("synthesizer/saved_models/synthesis.pt")
vocoder.load_model(Path("vocoder/saved_models/vocoder.pt"))

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1hoFzOhkAA_FMLxZujFma00Cj4cR1fmTL
To: /content/encoder/saved_models/encoder.pt
100% 17.1M/17.1M [00:00<00:00, 161MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=19DGdDixWUAzzF88aNd1FL3wQngtzdnry
From (redirected): https://drive.google.com/uc?id=19DGdDixWUAzzF88aNd1FL3wQngtzdnry&confirm=t&uuid=82b7a3cd-58f8-4e46-9a1c-acbc729e3ae4
To: /content/synthesizer/saved_models/synthesis.pt
100% 371M/371M [00:01<00:00, 205MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: Fu

In [7]:
# Function to clone voice
def clone_voice(text, source_audio):
  """
  Clone voice from source audio to generate text
  """
  # Preprocess the source audio
  preprocessed_wav = encoder.preprocess_wav(Path(source_audio))
  # Encode the source audio to get the embeding
  original_wav, sampling_rate = librosa.load(source_audio)
  embed = encoder.embed_utterance(preprocessed_wav)
  # Synthesize the speech
  specs = synthesizer.synthesize_spectrograms([text], [embed])
  generated_wav = vocoder.infer_waveform(specs[0])
  # Adjust the sample rate
  generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate) , mode="constant")
  return generated_wav

In [5]:
!wget https://www.voiptroubleshooter.com/open_speech/american/OSR_us_000_0037_8k.wav

--2024-07-06 06:52:29--  https://www.voiptroubleshooter.com/open_speech/american/OSR_us_000_0037_8k.wav
Resolving www.voiptroubleshooter.com (www.voiptroubleshooter.com)... 162.241.218.124
Connecting to www.voiptroubleshooter.com (www.voiptroubleshooter.com)|162.241.218.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 622080 (608K) [audio/x-wav]
Saving to: ‘OSR_us_000_0037_8k.wav’

OSR_us_000_0037_8k. 100%[===================>] 607.50K  3.16MB/s    in 0.2s    

2024-07-06 06:52:29 (3.16 MB/s) - ‘OSR_us_000_0037_8k.wav’ saved [622080/622080]



In [8]:
# Example Usage
text_to_clone = "Hello, how are you? This is a test to see the quality of the AI voice version"
source_audio = "/content/OSR_us_000_0037_8k.wav"

# Clone the voice
cloned_voice = clone_voice(text_to_clone, source_audio)

# Save or process the cloned voice further...
sampling_rate = 16000

# Convert the NumPy array to an audio file
sf.write("cloned_voice.wav", cloned_voice, sampling_rate)

# Play the cloned voice
Audio("cloned_voice.wav")


| Generating 1/1


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,




Done.

{| ████████████████ 76000/76800 | Batch Size: 8 | Gen Rate: 6.9kHz | }